## Linear Regression from scratch

In [ ]:
%matplotlib inline
import helper
import torch
import random

### Generate data

In [ ]:
def generate_synthetic_data(w, b, n):
    X = torch.normal(0, 1, (n, w.size()[0]))
    y = X @ w + b
    y += torch.normal(0, 0.01, y.size())
    return X,y.view(y.size(0),-1)

In [ ]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2

features, labels = generate_synthetic_data(true_w, true_b, 1000)

In [ ]:
helper.set_figsize((3.5, 2.5))
helper.plt.scatter(features[:, 1].numpy(), labels.numpy(), 1);

### Iterator on dataset

In [ ]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        yield features[i: min(i + batch_size, num_examples)], labels[i: min(i + batch_size, num_examples)]

batch_size = 10
for X,y in data_iter(batch_size, features, labels):
    print(X,y)
    break

### Linear Regression model, its params and loss function

In [ ]:
w = torch.normal(0, 0.01, (1,2), requires_grad=True)
b = torch.empty(1, requires_grad=True)

def model(X):
    # Simple linear regression model
    return X @ w.t() + b

def squared_loss(targets, preds):
    e = (targets-preds)**2
    return e.mean()


In [ ]:
batch_size=20
for epoch in range(1, 101):
    for X, y in data_iter(batch_size, features, labels):
        loss = squared_loss(y, model(X))
        loss.backward()
        with torch.no_grad():
            w -= w.grad * 1e-3
            b -= b.grad * 1e-3
            w.grad.zero_()
            b.grad.zero_()
    if epoch % 10 == 0:
        with torch.no_grad():
            train_l = loss = squared_loss(labels, model(features))
            print('epoch %d, loss %f' % (epoch, train_l.mean().numpy()))

In [ ]:
print('Error in estimating w', true_w - w[0])
print('Error in estimating b', true_b - b)